# 09: Metropolis Monte Carlo

This week we stick with Lennard-Jonesium as a material, and apply a famous alogrithm known as Metropolis Monte Carlo (MMC) or Markov Chain Monte Carlo (MCMC) to arrive at an ensemble of states for the system which is physical for conditions at finite temperature.  This approach can work for arbitrary ensembles, NVE, NVT, NPT, $\mu$PT$\ldots$ today we keep constant particle number N, volume V and temperature T.

Assuming classical Boltzmann statistics we can assign a probability to any given set of positions and momenta for our system:

$$
P(x, v) = \frac{ \exp\left[ -\beta (U(x) + K(v)) \right] }{Z}
$$

Where $\beta=1/k_BT$.  In Boltzmann statistics **only**, the kinetic energy $K$ factors out, so we can write the ratio of probabilities of two position-only states thus:


$$
P(x)/P(x') = \frac{ \exp\left[ -\beta U(x) \right] }{ \exp\left[ -\beta U(x') \right]  }
$$


$$
P(x)/P(x') = \exp\left[ -\beta ( U(x) - U(x') \right]
$$

Now, let us assume thermodynamic equilibrium.  A consequence of equilibrium is something called *detailed balance*:  equilibrium demands time symmetry in all things, so the probability to make a transition between two states, $x$ to $x'$, must be equal to the probability of making the same transition in the opposite direction,  $x'$ to $x$. We write the probability to be making the forward transition, using Bayes' rule, as $P(x)P(x\rightarrow x'|x)$ and the probability to be making the reverse transition as $P(x')P(x'\rightarrow x|x')$.  Thus:

$$
P(x)P(x\rightarrow x'|x) = P(x')P(x'\rightarrow x|x')
$$


$$
\frac{P(x'\rightarrow x|x')}{P(x\rightarrow x'|x)} = \frac{P(x)}{P(x')} 
$$

substituting:

$$
\frac{P(x\rightarrow x'|x)}{P(x'\rightarrow x|x')} = \exp\left[ -\beta ( U(x') - U(x) )\right]
$$

If we pick a pair of transition probabilities $P(x'\rightarrow x|x')$ and $P(x\rightarrow x'|x)$ which obey the above relation, then we are enforcing detailed balance in our system, and it follows that the probability distribution over positions $x$ should converge (after some equilibration) to sample the correct thermodynamic ensemble.

The easiest choice for the pair of transition probabilites such that the above relation is obeyed is to apply a different rule depending on if the transition is downhill $(U(x) > U(x'))$ or uphill $(U(x')>U(x))$.  If the transition is downhill we accept automatically; if it is uphill we apply the *Metropolis criterion:*

$$
P(\mathrm{accept}) = \exp\left[ -\beta ( U(x') - U(x) )\right]
$$

NB just because we accept downhill transitions automatically doesn't mean that the system always moves downhill! Detailed balance obliges us to sample reverse transitions $x'\rightarrow x$ proportionally to the amount of time that the system is in state $x'$, and *vice-versa*, so at finite $T$ there will always be some uphill flux.






Citation:


**Equation of State Calculations by Fast Computing Machines** *J. Chem. Phys.* 21, 1087 (1953)
Nicholas Metropolis, Arianna W. Rosenbluth, Marshall N. Rosenbluth, and Augusta H. Teller
*Los Alamos Scientific Laboratory, Los Alamos, New Mexico*








## Application Notes

Efficient application of MC depends on efficient calculation of energy **changes**.  Fortunately the local nature of the cut-n-shift LJ potential gives us access to an optimisation, when we move a particle then we can calculate a new neighbour list for it based on the efficient cell system that we already designed.  If we try to recalculate the whole system energy at every step, things will get bad quickly.

Sampling configurations for states is boring without any visualisation, but for now we are just going to monitor the potential energy, which should drop and then stabilise around some equilibrium value determined by the NVT parameters.   Later I hope we can track things like pressure, temperature, and measures of order in the system.


In [1]:
#include <stdio.h>
#include <stdlib.h>

//define an int array with information that
//we can use to change its size if it gets too big.
typedef struct xints_t {
    int *data;            //pointer to the stored data
    int  n_data_items;    //number of items in the array
    int  size_in_mem;     //size in memory (can be bigger but not smaller than n_data_items)
} XINTS_T;

//Define a (listable) structured type ATOM_T to hold info about a (classical) atom
//I've simplified the datatype to only contain info that is needed for today's problem
typedef struct atom_t {
    atom_t   *listNext;      //can add to a list of similar objects
    XINTS_T  *nebIds;        //keep an extendable array of int indices of neighbours
    int       myId;          //what am I?
    double    position[3];   
} ATOM_T; 


In [2]:

XINTS_T *create_xints(){
    XINTS_T *xints;
    
    xints = (XINTS_T *)malloc(sizeof(xints));
    xints->n_data_items = 0;
    xints->size_in_mem  = 8;
    xints->data = (int *)malloc(8*sizeof(int));
    
    return( xints );
}


In [3]:
void free_xints( XINTS_T *xints ){
    free(xints->data);
    free(xints);
}

In [4]:
void append_xint( XINTS_T *xints, int i ){
    if( xints->n_data_items >= xints->size_in_mem ){
        xints->size_in_mem *= 2; //realloc is expensive, so take more than we need.
        xints->data = (int *)realloc(xints->data, xints->size_in_mem*sizeof(int) );
    }
    xints->data[xints->n_data_items] = i; //save the new array member.
    xints->n_data_items             += 1;
}

In [5]:
void delete_xint( XINTS_T *xints, int remove_i ){
    
    //delete first occurence of 'remove_i' from the xints.
    int i, j;
    
    for( i = 0; i < xints->n_data_items; i++ ){
        if( xints->data[i] == remove_i ){
            for( j = i; j < xints->n_data_items - 1; j++ ){
                xints->data[j] = xints->data[j+1];
            }
            xints->n_data_items--;
            break;
        }
    }
}

In [6]:
ATOM_T *newAtom( double x, double y, double z, int id ){
    
    /* create an atom in a given position */
    
    ATOM_T *a;
    
    a = (ATOM_T *)malloc( sizeof(ATOM_T) );
    
    //info about the atom itself
    a->listNext     = NULL;
    a->myId         = id;
    a->position[0]  = x;
    a->position[1]  = y;
    a->position[2]  = z;
    
    //allocate some empty (but extendable) arrays to 
    //store connectivity information.
    a->nebIds       = create_xints();
    
    return( a );
}


In [7]:
void freeAtom( ATOM_T *a ){
    free( a->nebIds->data );
    free( a->nebIds);
    free( a );
}

In [8]:
ATOM_T **boxOfAtoms(int N_atoms, double box_L, int seed ){
    
    /*
     create some atoms and place them randomly in a box centred at the origin.
    */
    ATOM_T **atoms;
    int          i;
    double  half_L;
    
    half_L = box_L * 0.5;
    
    //init the random number generator, so that code is repeatable
    srand( seed );
    
    atoms = (ATOM_T **)malloc(N_atoms*sizeof(ATOM_T *));
    for( i = 0; i < N_atoms; i++ ){
        atoms[i] = newAtom( (rand()*(box_L/RAND_MAX)) - half_L,
                            (rand()*(box_L/RAND_MAX)) - half_L,
                            (rand()*(box_L/RAND_MAX)) - half_L, 
                             i );
        
    }
    return( atoms );
}

In [9]:
int image_int( int i, int N ){
    if( i >= N ) return ( i - N );
    if( i < 0  ) return ( i + N );
    return( i );
}

In [10]:
double check_contact( ATOM_T* a, ATOM_T* b, double box_L, double r_cut ){
    /* 
    check if two atoms are in "contact" (closer than distance r_cut)
    
    subject to periodic boundary conditions
    */
    double dx[3], r2, half_L;
    int    d;
    
    half_L = box_L * 0.5;
    for( d = 0; d < 3; d++ ){
      //displacement vector a to b.
      dx[d] = b->position[d] - a->position[d];
        
      //periodic imaging, now see nearest image.
      if     ( dx[d] >  half_L ) dx[d] -= box_L;
      else if( dx[d] < -half_L ) dx[d] += box_L;
    }
    
    r2 = dx[0]*dx[0] + dx[1]*dx[1] + dx[2]*dx[2]; 
    
    //return r if we are in contact, otherwise -1 (impossible r).
    if( r2 > r_cut*r_cut ) return( -1.0 );
    return( sqrt(r2) );
    
}

In [11]:
double U_LJpair( double *x1,  double *x2, double box_L, double r_cut ){
    /* 
    return Lennard-Jones interaction energy between two points
    */
    double dx[3], r2, inv_r6, inv_rcut6, dU, half_L;
    int    d;
    
    half_L = box_L * 0.5;
    for( d = 0; d < 3; d++ ){
      //displacement vector a to b.
      dx[d] = x2[d] - x1[d];
        
      //periodic imaging, now see nearest image.
      if     ( dx[d] >  half_L ) dx[d] -= box_L;
      else if( dx[d] < -half_L ) dx[d] += box_L;
    }
    
    r2 = dx[0]*dx[0] + dx[1]*dx[1] + dx[2]*dx[2]; 
    
    if( r2 > r_cut*r_cut ) return( 0.0 );
    inv_r6 = 1. / ( r2*r2*r2 );
    
    dU    = inv_r6*inv_r6 - inv_r6;
    
    //JTB lazy coding: just evaluate the shift every time the function is called
    inv_rcut6  = 1./r_cut;
    inv_rcut6 *= inv_rcut6;
    inv_rcut6  = inv_rcut6 * inv_rcut6 * inv_rcut6;
    dU        -= (inv_rcut6*inv_rcut6 - inv_rcut6);
    
    return( dU );
    
}

In [12]:
double U_LJsys( ATOM_T **atoms, int N_atoms, double box_L, double r_cut ){
    /* 
    return total Lennard-Jones interaction energy
    */
    double dx[3], r2, inv_r6, inv_rcut6, dU, half_L;
    int    i, j, n, d;
    double U_LJ = 0.0;
    
    //for each atom
    for( i = 0; i < N_atoms; i++ ){
        
        //for each neighbour of that atom
        for( j = 0; j < atoms[i]->nebIds->n_data_items; j++){
            
            //id of jth neighbour
            n     = atoms[i]->nebIds->data[j];
            
            if( n > i ){          
               U_LJ += U_LJpair( atoms[i]->position,  atoms[n]->position, box_L, r_cut );
            }
        }
    }
    return( U_LJ );
}

In [13]:
void assignCellIndex( double *posn, int *ix, int *iy, int *iz, double box_L, double r_cell ){
    /* assign cell indices for points on [-0.5*box_L.. 0.5*box_L) 
    
       This is done by integer rounding-down, assumes that the 
       box is centred at the origin and that all particles are
       in the box.   [-L/2 <= x,y,z < L/2)
    */
    double half_L;
    
    half_L = box_L * 0.5;
    
    //write to the output variables provided by the pointers ix, iy, iz
   *ix = (int)((posn[0]+half_L)/r_cell);
   *iy = (int)((posn[1]+half_L)/r_cell);
   *iz = (int)((posn[2]+half_L)/r_cell);
}

In [14]:
ATOM_T **buildCellList( ATOM_T **atoms, int N, double L, double rcut ){
    
    ATOM_T **cells;
    
    double r_cell, rcontact;
    int    Ncells_x, Ncells_x2, Ncells, i;
    int    ii, jj, kk;
    
    //how big should the cells be, and how many do we need?
    Ncells_x  = (int)(L / rcut);      //count the number of cells in each direction
    if( Ncells_x < 3 )Ncells_x = 3;   //minimum is three
    r_cell    = L / Ncells_x;         //length per cell
    Ncells_x2 = Ncells_x * Ncells_x;
    Ncells    = Ncells_x * Ncells_x * Ncells_x;
    
    //allocate space for the cell array
    cells = (ATOM_T**)malloc(Ncells*sizeof(ATOM_T*));
    for(i = 0; i < Ncells; i++) cells[i] = NULL; //cells start off empty
    
    //assign atoms to cells
    for(i = 0; i < N; i++){
        //get cell indices ii, jj, kk
        assignCellIndex( atoms[i]->position, &ii, &jj, &kk, L, r_cell );
        
        //add the atom to the cell it belongs to (ii + jj*Ncells_x + kk*Ncells_x2)
        atoms[i]->listNext                 = cells[ii+jj*Ncells_x+kk*Ncells_x2];
        cells[ii+jj*Ncells_x+kk*Ncells_x2] = atoms[i];
        
    }
    return ( cells );
}

In [15]:
void buildNeighbourLists(  ATOM_T **atoms, int N, double L, double rcut, ATOM_T **cells ) {
    
    ATOM_T *a, *b;
    
    //duplicate some convenience variables that depend on rcut and L
    double r_cell, rcontact;
    int    Ncells_x, Ncells_x2, Ncells, i;
    int    ii, jj, kk;
    int    iii, jjj, kkk,  i0, j0, k0;
    
    Ncells_x  = (int)(L / rcut);      //count the number of cells in each direction
    if( Ncells_x < 3 )Ncells_x = 3;   //minimum is three
    r_cell    = L / Ncells_x;         //length per cell
    Ncells_x2 = Ncells_x * Ncells_x;
    Ncells    = Ncells_x * Ncells_x * Ncells_x;
    
    /*
    *  check contacts and build neighbour xarrays
    */
    //loop over all cells.
    for(i = 0; i < Ncells; i++){
        
        //i0,j0,k0 indices of this cell in 3D
        i0 =       i % Ncells_x;
        j0 =      (i % Ncells_x2) / Ncells_x;
        k0 = (int) i / Ncells_x2;
        
        //a is the first atom in the cell, or NULL if no atoms
        a = cells[i];
        while( a != NULL ){
            
            //check all 27 neighbouring cells
            for( ii = -1; ii <= 1; ii++ ){
                iii = image_int( i0+ii, Ncells_x );
            for( jj = -1; jj <= 1; jj++ ){
                jjj = image_int( j0+jj, Ncells_x );
            for( kk = -1; kk <= 1; kk++ ){
                kkk = image_int( k0+kk, Ncells_x );
                b = cells[ iii + jjj*Ncells_x + kkk*Ncells_x2 ];
                if( a == b ) continue;
                while( b ){
                   if( b != a ){
                       if( check_contact(a, b, L, rcut) >= 0 ){
                           
                           //new code for this week: add the atom b
                           //to the neighbour xarray of a.
                           append_xint( a->nebIds, b->myId );
                               
                       }
                   }
                   b = b->listNext;  //keep looking in this neighbour cell
                }
            }
            }
            }
            a = a->listNext; //keep looking in the main cell.
        }
    }
}

In [16]:
double moveParticle(  ATOM_T **atoms, int N, double L, double rcut, ATOM_T **cells, int move_i, double* new_x ) {
    
    ATOM_T *a, *b, *bprev;
    
    //duplicate some convenience variables that depend on rcut and L
    double r_cell, rcontact;
    int    Ncells_x, Ncells_x2, Ncells, i, j, n, already_nebs;
    int    ii, jj, kk;
    int    new_icell, old_icell;
    int    iii, jjj, kkk,  i0, j0, k0;
    
    Ncells_x  = (int)(L / rcut);      //count the number of cells in each direction
    if( Ncells_x < 3 )Ncells_x = 3;   //minimum is three
    r_cell    = L / Ncells_x;         //length per cell
    Ncells_x2 = Ncells_x * Ncells_x;
    Ncells    = Ncells_x * Ncells_x * Ncells_x;
    
    //working variables for the MC move
    double old_Uat, new_Uat, delta_Uat;
    
    //get a pointer to the atom to be moved
    a = atoms[move_i];
    
    //Current neighbourhood energy interactions
    //for each neighbour of that atom
    old_Uat    = 0.0;
    for( j = 0; j < a->nebIds->n_data_items; j++){
        
            //id of jth neighbour
            n = a->nebIds->data[j];
           
            //Add LJ energy contribution.
            old_Uat += U_LJpair( a->position,  atoms[n]->position, L, rcut );
        
            //clean atom a from the neighbour list of b
            delete_xint( atoms[n]->nebIds, move_i );
        
    }
    
    //refresh the neighbour list of a and build a new one for the new position.
    free_xints( a->nebIds );
    a->nebIds = create_xints();
    
    //what cell was atom 'a' in?
    assignCellIndex( a->position, &i0, &j0, &k0, L, r_cell );    
    old_icell = i0 + j0*Ncells_x + k0*Ncells_x2;
    
    //what cell is atom 'a' moving to?
    assignCellIndex( new_x, &i0, &j0, &k0, L, r_cell );    
    new_icell = i0 + j0*Ncells_x + k0*Ncells_x2;
    
    //restitch the cell pointers
    b     = cells[old_icell];
    bprev = NULL;
    while( b ){
        if( a == b ){
            if( bprev ) bprev->listNext  = a->listNext;
            else        cells[old_icell] = a->listNext;
            break;
        }
        bprev = b;
        b     = b->listNext;
    }
    
    //add to new cell (head of list).
    a->listNext = cells[new_icell];
    cells[new_icell] = a;
    
    //update position
    for(i = 0; i < 3; i++ ) a->position[i] = new_x[i];
     
    //energy calculation: check all 27 neighbouring cells
    new_Uat = 0.0;
    for( ii = -1; ii <= 1; ii++ ){
         iii = image_int( i0+ii, Ncells_x );
         for( jj = -1; jj <= 1; jj++ ){
            jjj = image_int( j0+jj, Ncells_x );
            for( kk = -1; kk <= 1; kk++ ){
                kkk = image_int( k0+kk, Ncells_x );
                    
                //neighbour atom 'b'
                b = cells[ iii + jjj*Ncells_x + kkk*Ncells_x2 ];
                while( b ){
                    
                       //don't compare an atom to itself.
                       if( a == b ){
                           b = b->listNext;
                           continue;
                       }
                        
                       //get the energy change
                       delta_Uat = U_LJpair( a->position,  b->position, L, rcut );
                    
                       rcontact = check_contact( a, b, L, rcut );
                    
                       if( rcontact != 0.0 ){
                           
                           //create the new neighbour list as we go
                           append_xint( a->nebIds, b->myId );
                           
                           //need to check that atom a is not already 
                           //in b's neighbour list before we add it.
                           already_nebs = 0;
                           for( n = 0; n < b->nebIds->n_data_items; n++ ){
                               if( move_i == b->nebIds->data[n] ) {
                                   already_nebs = 1;
                                   break;
                               }
                           }
                           if( already_nebs == 0 )append_xint( b->nebIds, move_i );
                           
                           
                           new_Uat += delta_Uat;
                       }
                       b = b->listNext;  //keep looking in this neighbour cell
                }
            }
        }
    }
    return( new_Uat - old_Uat );
    
}

In [48]:
int test_MC(int N_atoms, double box_L, int seed, double r_cut, int N_sweeps, double beta){
    
    /*
      Test code to:
      
      run a metropolis monte carlo.
      
      **STUB CODE : INCOMPLETE: *YOU* HAVE TO GET IT WORKING.
      
    */
    ATOM_T **atoms, **cells;
    int      i, j, n, d, i_sweep, n_sweeps;
    int      i_atom;
    double   *pU;
    double   new_x[3], old_x[3], half_L, dU, U_total, U_start, U_end, sum_dU, U_old, U_new, U_avg, P_accept, deviation;
    
    half_L = box_L * 0.5;
    
    srand( seed );
    
    U_total = 0.;
    U_avg = 0.;
    
    
    // allocate memory for U values
    pU = (double *)malloc(N_sweeps*sizeof(double));
    
    //place a bunch of atoms randomly in a box of size box_L
    atoms = boxOfAtoms( N_atoms, box_L, seed );
    
    //build a cell list so we can easily find neighbours of atoms
    cells = buildCellList( atoms, N_atoms, box_L, r_cut);
    
    //give each atom a list of its neighbours, stored inside the ATOM_T
    //as an extensible array.
    buildNeighbourLists( atoms, N_atoms, box_L, r_cut, cells );
    
    //for debug / checking:
    U_start = U_LJsys( atoms, N_atoms, box_L, r_cut );
    U_old = U_start;
    
    /*******************MMC*/
    sum_dU = 0.0;
    for( i_sweep = 0; i_sweep < N_sweeps; i_sweep++){
        
        //each 'sweep' move each atom once.
        for(i_atom = 0; i_atom < N_atoms; i_atom ++ ){
         
            //pick a random position
            for( d = 0; d < 3; d++ ){
                old_x[d] = atoms[i_atom]->position[d];
                new_x[d] = (rand()*(box_L/RAND_MAX)) - half_L; // insert propose move here + keep counter of success
            }

            //what is the energy change?
            dU = moveParticle( atoms, N_atoms, box_L, r_cut, cells, i_atom, new_x );
                        
            // update new energy value
            U_new = U_old + dU;

            // Metropolis Criterion
            if (dU <= 0){
                pU[i_sweep] = U_new; // store the energy of the run
            }
            else{
                P_accept = (double)rand()/RAND_MAX; // probability between 0 and 1
                if (P_accept > exp(- beta * dU) ){ // not accepted
                    // revert changes made to the system
                    U_new = U_old;
                    //pU[i_sweep] += U_new;
                    moveParticle( atoms, N_atoms, box_L, r_cut, cells, i_atom, old_x ); 
                }
                else { pU[i_sweep] = U_new; }
            }
            
            // track energy change
            sum_dU += U_new - U_old;

            // invert assignment for the next loop
            U_old = U_new;  
            U_new = 0;
        }
        // pU[i_sweep] = U_new;
        U_total += pU[i_sweep];
        U_avg = U_total/(double)N_sweeps/(double)N_atoms;
    }
    
    U_end = U_LJsys( atoms, N_atoms, box_L, r_cut );
    deviation = fabs((U_end - U_start) - sum_dU);

    printf("For beta = %f\n\n",beta);
    printf("U went from %lf to %lf after sweeps,\ndelta should be %lf, delta was: %lf, deviation of: %e\n",
                U_start, U_end, U_end - U_start, sum_dU, deviation);
    printf("Total energy = %f \nAverage Energy of the system is: %f\n\n--------------------------------\n", U_total, U_avg);
    
    
    /**********************/
    
    //clean up.
    free( atoms ); //this releases the array of double-pointers to atoms.
    free( cells ); //a cell is just a pointer to the first atom in a list, we can free directly.
    
    return( EXIT_SUCCESS );
}


In [50]:
test_MC( 100, 15.0, 1337, 4.0, 30, 0.0);
test_MC( 100, 15.0, 1337, 4.0, 30, 0.5);
test_MC( 100, 15.0, 1337, 4.0, 30, 1.0);
test_MC( 100, 15.0, 1337, 4.0, 30, 2.0);
test_MC( 100, 15.0, 1337, 4.0, 30, INFINITY);

For beta = 0.000000

U went from 50.382096 to 750575.099033 after sweeps,
delta should be 750524.716936, delta was: 750524.717693, deviation of: 7.562616e-04
Total energy = 864139276252.432129 
Average Energy of the system is: 288046425.417477

--------------------------------
For beta = 0.500000

U went from 50.382096 to -4.972412 after sweeps,
delta should be -55.354508, delta was: -55.354508, deviation of: 9.237056e-14
Total energy = -83.180538 
Average Energy of the system is: -0.027727

--------------------------------
For beta = 1.000000

U went from 50.382096 to -4.206872 after sweeps,
delta should be -54.588968, delta was: -54.588968, deviation of: 0.000000e+00
Total energy = -110.505344 
Average Energy of the system is: -0.036835

--------------------------------
For beta = 2.000000

U went from 50.382096 to -4.391872 after sweeps,
delta should be -54.773968, delta was: -54.773968, deviation of: 1.421085e-13
Total energy = -146.957488 
Average Energy of the system is: -0.04898

OK great I have given you some code to 

1) Create a box of atoms

2) See if two atoms are in contact

3) Make an extensible array of atom ids showing the neighbours of each atom.

Before I do the whole exercise I am going to stop coding and ask you to take over.


## Assignment, week 09: Warming Lennard Jonesium

The easiest observable to sanity-check the MMC algorithm is the potential energy as a function of temperature, obviously this should go up as the system gets hotter.

(1) Implement the MMC algorithm

(2) Run MMC for multiple temperatures.  We should keep the number of particles and the other parameters constant, and report potential energy scaled per particle $U/N$.

$$
N = 100
$$

$$
L = 15.0
$$

$$
r_{cut} = 4.0
$$

$$
\mathrm{seed} = 1337
$$

I'm going to ask you to use the "leet" number as a seed so that all code is repeatable.


$Tk_B$ = 0.0, 0.5, 1.0, 2.0.  

I'm going to leave it to you to verify how many steps are needed to converge to the equilibrium distribution, and also to decide how much time you need to sample for in order to get a good average for $<U/N>_{|T}$.  I'll give 110% to whoever gets closest to my reference values (that I'll use a proper computer to find, overnight); everyone else who gets the exercise correct and has sane values will get 100%.


